# Reproduce CESM2-MARBL Figures
Within this example, we will reproduce figures from the CESM2-MARBL Paper, including
* Table of globally integrated values from different experiments
* Zonal average plots
* 2D map of yearly averages, epoch means from some selection dictionary

In [2]:
import os

os.environ["PREFECT__FLOWS__CHECKPOINTING"] = "True"

import tempfile
import time

import numpy as np
import prefect
import xarray as xr
import xcollection as xc
from distributed import Client
from ncar_jobqueue import NCARCluster
from prefect import Flow, Parameter, task
from xpersist import CacheStore, XpersistResult

In [3]:
import intake

In [4]:
store = CacheStore(f'{tempfile.gettempdir()}/marbl-example-cache')
store

CacheStore(path='/glade/scratch/mgrover/marbl-example-cache', readonly=False, on_duplicate_key=<DuplicateKeyEnum.skip: 'skip'>, storage_options={})

In [5]:
cluster = NCARCluster()
cluster.scale(20)
client = Client(cluster)
client

/glade/work/mgrover/miniconda3/envs/esds-funnel-docs/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43196 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mgrover/proxy/43196/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mgrover/proxy/43196/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.12.206.54:37467,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/mgrover/proxy/43196/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Functions to help with the Analysis

In [6]:
def global_mean(
    ds,
    horizontal_dims,
    area_field,
    land_sea_mask,
    normalize=True,
    include_ms=False,
    region_mask=None,
    time_dim="year",
):
    """
    Compute the global mean on some dataset
    Return computed quantity in conventional units.
    """

    compute_vars = [v for v in ds if time_dim in ds[v].dims and horizontal_dims == ds[v].dims[-2:]]

    other_vars = list(set(ds.variables) - set(compute_vars))

    if include_ms:
        surface_mask = ds[area_field].where(ds[land_sea_mask] > 0).fillna(0.0)
    else:
        surface_mask = ds[area_field].where(ds.REGION_MASK > 0).fillna(0.0)

    if region_mask is not None:
        surface_mask = surface_mask * region_mask

    masked_area = {v: surface_mask.where(ds[v].notnull()).fillna(0.0) for v in compute_vars}

    with xr.set_options(keep_attrs=True):

        dso = xr.Dataset({v: (ds[v] * masked_area[v]).sum(horizontal_dims) for v in compute_vars})
        if normalize:
            dso = xr.Dataset(
                {v: dso[v] / masked_area[v].sum(horizontal_dims) for v in compute_vars}
            )

        return xr.merge([dso, ds[other_vars]]).drop(
            [c for c in ds.coords if ds[c].dims == horizontal_dims]
        )


def zonal_mean(
    da_in,
    grid,
    lat_axis=None,
    lat_field='geolat',
    ydim='yh',
    xdim='xh',
    area_field='area_t',
    region_mask=None,
):
    """Calculate a zonal average from some model on xarray.DataArray

    Parameters
    ----------

    da_in : xarray.DataArray
       DataArray to calculate a zonal average from. This should be your data variable

    grid : xarray.Dataset
       Grid with the latitude, area field, and latitude axis (if needed), matching dims of da_in

    lat_axis : xarray.DataArray
       Latitude axis to use for latitude bins

    lat_field : string
       Name of the latitude field to use

    ydim : string
       Name of y-dimension

    xdim : string
       Name of x-dimension

    area_field : string
       Field to use for the area values, used for weighting

    Returns
    -------
    da_out : xarray.DataArray
       Resultant zonally averaged field, with the same input name and a new latitude bin axis
    """

    # If not provided a latitude axis, use the y-axis
    if lat_axis is None:
        lat_axis = grid[ydim]

    area = grid[area_field].broadcast_like(da_in).where(da_in > -9999)
    lat_2d = grid[lat_field]

    if region_mask is not None:
        da_in = da_in.where(region_mask > 0)
        area = area * region_mask.where(region_mask > 0)
        lat_2d = lat_2d.where(region_mask > 0)

    # Create the latitude bins using the lat_axis data array
    bins = lat_axis.values

    # Calculate the numerator
    histVolCoordDepth = histogram(
        lat_2d.broadcast_like(area).where(~np.isnan(area)),
        bins=[bins],
        weights=area,
        dim=[ydim, xdim],
    )

    # Calculate the denominator
    histTVolCoordDepth = histogram(
        lat_2d.broadcast_like(area).where(~np.isnan(area)),
        bins=[bins],
        weights=(area * da_in).fillna(0),
        dim=[ydim, xdim],
    )

    if region_mask is not None:
        histRegionVolCoordDepth = histogram(
            lat_2d.broadcast_like(area).where(~np.isnan(area)),
            bins=[bins],
            weights=(area * region_mask).fillna(0),
            dim=[ydim, xdim],
        )

    da_out = (histTVolCoordDepth / histVolCoordDepth).rename(da_in.name)

    # Return the zonal average, renaming the variable to the variable in
    return da_out


def yearly_mean(ds):
    """
    weight by days in each month
    """
    # Determine the month length
    month_length = ds.time.dt.days_in_month

    # Calculate the weights
    wgts = month_length.groupby("time.year") / month_length.groupby("time.year").sum()

    # Make sure the weights in each year add up to 1
    np.testing.assert_allclose(wgts.groupby("time.year").sum(xr.ALL_DIMS), 1.0)

    # Subset our dataset for our variable
    obs = ds

    # Setup our masking for nan values
    cond = obs.isnull()
    ones = xr.where(cond, 0.0, 1.0)

    # Calculate the numerator
    obs_sum = (obs * wgts).resample(time="AS").sum(dim="time")

    # Calculate the denominator
    ones_out = (ones * wgts).resample(time="AS").sum(dim="time")

    # Return the weighted average
    return obs_sum / ones_out


def time_mean(ds):
    return ds.mean(dim='time')

## Tasks

In [7]:
@task
def read_catalog(path, multivar_row=False):
    if multivar_row:
        read_csv_kwargs = {"converters": {"variables": ast.literal_eval}}
    else:
        read_csv_kwargs = None
    return intake.open_esm_datastore(path, read_csv_kwargs=read_csv_kwargs)


@task
def subset_catalog(intake_esm_catalog, search_dict):
    return intake_esm_catalog.search(**search_dict)


@task
def load_catalog(intake_esm_catalog, cdf_kwargs={}):
    return intake_esm_catalog.to_collection(cdf_kwargs=cdf_kwargs)


@task
def get_collection_values(collection):
    return list(collection.values())


@task
def get_collection_keys(collection):
    return list(collection.keys())


@task(
    target="{flow_name}-{task_name}-{key}.zarr",
    result=XpersistResult(store, serializer="xarray.zarr", serializer_dump_kwargs={"mode": "w"}),
)
def annual_average(ds, key):
    return yearly_mean(ds)


@task(
    target="{flow_name}-{task_name}-{key}.zarr",
    result=XpersistResult(store, serializer="xarray.zarr", serializer_dump_kwargs={"mode": "w"}),
)
def time_average(ds, key):
    return time_mean(ds)

## Setup the Different Flows

### Yearly Mean

This Flow ***does not work***

In [8]:
with Flow('yearly_mean') as yearly_mean_flow:
    collection_path = Parameter(
        'collection_path',
        default='/glade/campaign/cesm/development/omwg/projects/MOMMARBL_vs_POPECO/catalog/MOMvsPOP.json',
    )
    multivar_row = Parameter('multi_var_row', default=False)
    search_dict = Parameter('search_dict', default={})
    cdf_kwargs = Parameter('cdf_kwargs', default=None)

    # Load the intake-esm catalog into memory
    data_catalog = read_catalog(collection_path, multivar_row)

    # Subset the catalog
    data_catalog_subset = subset_catalog(data_catalog, search_dict)

    # Convert to catalog to an xcollection
    collection = load_catalog(data_catalog_subset, cdf_kwargs=cdf_kwargs)

    values = get_collection_values(collection)
    keys = get_collection_keys(collection)

    # Apply the annual average operator
    yearly_average_collection = annual_average.map(
        list(collection.values()), list(collection.keys())
    )

AttributeError: 'FunctionTask' object has no attribute 'values'

In [9]:
with Flow('time_mean') as time_mean_flow:
    collection_path = Parameter(
        'collection_path',
        default='/glade/campaign/cesm/development/omwg/projects/MOMMARBL_vs_POPECO/catalog/MOMvsPOP.json',
    )
    multivar_row = Parameter('multi_var_row', default=False)
    search_dict = Parameter('search_dict', default={})
    cdf_kwargs = Parameter('cdf_kwargs', default=None)

    # Load the intake-esm catalog into memory
    data_catalog = read_catalog(collection_path, multivar_row)

    # Subset the catalog
    data_catalog_subset = subset_catalog(data_catalog, search_dict)

    # Convert to catalog to an xcollection
    collection = load_catalog(data_catalog_subset, cdf_kwargs=cdf_kwargs)

    values = get_collection_values(collection)
    keys = get_collection_keys(collection)

    # Apply the annual average operator
    yearly_average_collection = time_average.map(values, keys)

In [29]:
time_mean_flow.run(search_dict={'stream': 'pop.h', 'variable': 'TEMP'}, cdf_kwargs={})

[2021-12-20 15:24:54-0700] INFO - prefect.FlowRunner | Beginning Flow run for 'time_mean'
[2021-12-20 15:24:54-0700] INFO - prefect.TaskRunner | Task 'search_dict': Starting task run...
[2021-12-20 15:24:54-0700] INFO - prefect.TaskRunner | Task 'search_dict': Finished task run for task with final state: 'Success'
[2021-12-20 15:24:54-0700] INFO - prefect.TaskRunner | Task 'cdf_kwargs': Starting task run...
[2021-12-20 15:24:54-0700] INFO - prefect.TaskRunner | Task 'cdf_kwargs': Finished task run for task with final state: 'Success'
[2021-12-20 15:24:54-0700] INFO - prefect.TaskRunner | Task 'multi_var_row': Starting task run...
[2021-12-20 15:24:54-0700] INFO - prefect.TaskRunner | Task 'multi_var_row': Finished task run for task with final state: 'Success'
[2021-12-20 15:24:54-0700] INFO - prefect.TaskRunner | Task 'collection_path': Starting task run...
[2021-12-20 15:24:54-0700] INFO - prefect.TaskRunner | Task 'collection_path': Finished task run for task with final state: 'Succe

[2021-12-20 15:24:56-0700] INFO - prefect.TaskRunner | Task 'load_catalog': Finished task run for task with final state: 'Success'
[2021-12-20 15:24:56-0700] INFO - prefect.TaskRunner | Task 'get_collection_values': Starting task run...
[2021-12-20 15:24:56-0700] INFO - prefect.TaskRunner | Task 'get_collection_values': Finished task run for task with final state: 'Success'
[2021-12-20 15:24:56-0700] INFO - prefect.TaskRunner | Task 'get_collection_keys': Starting task run...
[2021-12-20 15:24:56-0700] INFO - prefect.TaskRunner | Task 'get_collection_keys': Finished task run for task with final state: 'Success'
[2021-12-20 15:24:56-0700] INFO - prefect.TaskRunner | Task 'time_average': Starting task run...
[2021-12-20 15:24:56-0700] INFO - prefect.TaskRunner | Task 'time_average': Finished task run for task with final state: 'Mapped'
[2021-12-20 15:24:56-0700] INFO - prefect.TaskRunner | Task 'time_average[0]': Starting task run...
[2021-12-20 15:25:41-0700] INFO - prefect.TaskRunner |

<Success: "All reference tasks succeeded.">

In [10]:
store.keys()

['time_mean-time_average-ocn.pop.h.pop_control.zarr',
 'time_mean-time_average-ocn.pop.h.pop_no_mcog.zarr']

In [11]:
store.get('time_mean-time_average-ocn.pop.h.pop_control.zarr')

<xarray.Dataset>
Dimensions:                 (nlat: 384, nlon: 320, z_t: 60, z_w: 60, lat_aux_grid: 395, moc_comp: 3, moc_z: 61, transport_comp: 5, transport_reg: 2, z_t_150m: 15, z_w_bot: 60, z_w_top: 60)
Coordinates:
    TLAT                    (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    TLONG                   (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    ULAT                    (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    ULONG                   (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
  * lat_aux_grid            (lat_aux_grid) float32 -79.49 -78.95 ... 89.47 90.0
  * moc_z                   (moc_z) float32 0.0 1e+03 2e+03 ... 5.25e+05 5.5e+05
  * z_t                     (z_t) float32 500.0 1.5e+03 ... 5.125e+05 5.375e+05
  * z_t_150m                (z_t_150m) float32 500.0 1.5e+03 ... 1.45e+04
  * z_w                     (z_w) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
  * z_w_bot                 (z_w_bot) float32 1e+03 2e+03 ... 5.25e+05 5.5e+05
  * z_w_top                 (z_w_top) float32 0.0 1e+03 2e+03 ... 5e+05 5.25e+05
Dimensions without coordinates: nlat, nlon, moc_comp, transport_comp, transport_reg
Data variables: (12/54)
    ANGLE                   (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    ANGLET                  (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    DXT                     (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    DXU                     (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    DYT                     (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    DYU                     (nlat, nlon) float64 dask.array<chunksize=(384, 320), meta=np.ndarray>
    ...                      ...
    sflux_factor            float64 ...
    sound                   float64 ...
    stefan_boltzmann        float64 ...
    transport_components    (transport_comp) |S384 dask.array<chunksize=(5,), meta=np.ndarray>
    transport_regions       (transport_reg) |S384 dask.array<chunksize=(2,), meta=np.ndarray>
    vonkar                  float64 ...